In [35]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.layers import InputLayer, Dense

import matplotlib.pyplot as plt

import math

from time import time

from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [3]:
df = pd.read_csv("/data/kddcup.data", header=None)

In [4]:
columns = [f.split(":")[0] for f in """
duration: continuous.
protocol_type: symbolic.
service: symbolic.
flag: symbolic.
src_bytes: continuous.
dst_bytes: continuous.
land: symbolic.
wrong_fragment: continuous.
urgent: continuous.
hot: continuous.
num_failed_logins: continuous.
logged_in: symbolic.
num_compromised: continuous.
root_shell: continuous.
su_attempted: continuous.
num_root: continuous.
num_file_creations: continuous.
num_shells: continuous.
num_access_files: continuous.
num_outbound_cmds: continuous.
is_host_login: symbolic.
is_guest_login: symbolic.
count: continuous.
srv_count: continuous.
serror_rate: continuous.
srv_serror_rate: continuous.
rerror_rate: continuous.
srv_rerror_rate: continuous.
same_srv_rate: continuous.
diff_srv_rate: continuous.
srv_diff_host_rate: continuous.
dst_host_count: continuous.
dst_host_srv_count: continuous.
dst_host_same_srv_rate: continuous.
dst_host_diff_srv_rate: continuous.
dst_host_same_src_port_rate: continuous.
dst_host_srv_diff_host_rate: continuous.
dst_host_serror_rate: continuous.
dst_host_srv_serror_rate: continuous.
dst_host_rerror_rate: continuous.
dst_host_srv_rerror_rate: continuous.
""".split("\n") if len(f)>0]

columns.append("Category")
print(columns)

['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'Category']


In [5]:
df.columns = columns

In [6]:
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Category
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898431 entries, 0 to 4898430
Data columns (total 42 columns):
duration                       int64
protocol_type                  object
service                        object
flag                           object
src_bytes                      int64
dst_bytes                      int64
land                           int64
wrong_fragment                 int64
urgent                         int64
hot                            int64
num_failed_logins              int64
logged_in                      int64
num_compromised                int64
root_shell                     int64
su_attempted                   int64
num_root                       int64
num_file_creations             int64
num_shells                     int64
num_access_files               int64
num_outbound_cmds              int64
is_host_login                  int64
is_guest_login                 int64
count                          int64
srv_count                      in

In [8]:
df.Category.value_counts()

smurf.              2807886
neptune.            1072017
normal.              972781
satan.                15892
ipsweep.              12481
portsweep.            10413
nmap.                  2316
back.                  2203
warezclient.           1020
teardrop.               979
pod.                    264
guess_passwd.            53
buffer_overflow.         30
land.                    21
warezmaster.             20
imap.                    12
rootkit.                 10
loadmodule.               9
ftp_write.                8
multihop.                 7
phf.                      4
perl.                     3
spy.                      2
Name: Category, dtype: int64

In [27]:
y = np.where(df.Category == "normal.", 0, 1)

In [17]:
X = df.drop(columns=["Category", "flag", "service"])
X.head()

,duration,protocol_type,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,tcp,215,45076,0,0,0,0,0,1,...,0,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0
1,0,tcp,162,4528,0,0,0,0,0,1,...,1,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0
2,0,tcp,236,1228,0,0,0,0,0,1,...,2,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0
3,0,tcp,233,2032,0,0,0,0,0,1,...,3,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0
4,0,tcp,239,486,0,0,0,0,0,1,...,4,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0


In [18]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898431 entries, 0 to 4898430
Data columns (total 39 columns):
duration                       int64
protocol_type                  object
src_bytes                      int64
dst_bytes                      int64
land                           int64
wrong_fragment                 int64
urgent                         int64
hot                            int64
num_failed_logins              int64
logged_in                      int64
num_compromised                int64
root_shell                     int64
su_attempted                   int64
num_root                       int64
num_file_creations             int64
num_shells                     int64
num_access_files               int64
num_outbound_cmds              int64
is_host_login                  int64
is_guest_login                 int64
count                          int64
srv_count                      int64
serror_rate                    float64
srv_serror_rate                fl

In [21]:
X = pd.get_dummies(X, drop_first=True)

In [13]:
df.protocol_type.value_counts()

icmp    2833545
tcp     1870598
udp      194288
Name: protocol_type, dtype: int64

In [15]:
df.service.value_counts()

ecr_i          2811660
private        1100831
http            623091
smtp             96554
other            72653
domain_u         57782
ftp_data         40697
eco_i            16338
finger            6891
urp_i             5378
ftp               5214
telnet            4277
ntp_u             3833
auth              3382
pop_3             1981
time              1579
domain            1113
Z39_50            1078
gopher            1077
mtp               1076
ssh               1075
whois             1073
remote_job        1073
rje               1070
imap4             1069
link              1069
ctf               1068
name              1067
supdup            1060
echo              1059
                ...   
vmnet             1053
sql_net           1052
iso_tsap          1052
netbios_dgm       1052
csnet_ns          1051
shell             1051
hostnames         1050
klogin            1050
bgp               1047
login             1045
printer           1045
exec              1045
http_443   

In [16]:
df.flag.value_counts()

SF        3744328
S0         869829
REJ        268874
RSTR         8094
RSTO         5344
SH           1040
S1            532
S2            161
RSTOS0        122
OTH            57
S3             50
Name: flag, dtype: int64

In [20]:
X.isnull().sum(axis = 0)

duration                       0
protocol_type                  0
src_bytes                      0
dst_bytes                      0
land                           0
wrong_fragment                 0
urgent                         0
hot                            0
num_failed_logins              0
logged_in                      0
num_compromised                0
root_shell                     0
su_attempted                   0
num_root                       0
num_file_creations             0
num_shells                     0
num_access_files               0
num_outbound_cmds              0
is_host_login                  0
is_guest_login                 0
count                          0
srv_count                      0
serror_rate                    0
srv_serror_rate                0
rerror_rate                    0
srv_rerror_rate                0
same_srv_rate                  0
diff_srv_rate                  0
srv_diff_host_rate             0
dst_host_count                 0
dst_host_s

In [23]:
training_size = int(X.shape[0] * 0.7)
training_size

3428901

In [28]:
X_train, y_train = X[:training_size], y[:training_size]
X_test, y_test = X[training_size:], y[training_size:]

In [29]:
pd.Series(y_train).value_counts()/len(y_train)

1    0.783221
0    0.216779
dtype: float64

In [30]:
pd.Series(y_train).value_counts()/len(y_train)

1    0.783221
0    0.216779
dtype: float64

In [33]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
n_x = X_train.shape[1]

tf.set_random_seed(1)
np.random.seed(1)

tensor_board = keras.callbacks.TensorBoard(log_dir="/tmp/tf/logs/%d" % time())

model = keras.Sequential()
model.add(InputLayer((n_x, )))
model.add(Dense(units=1, activation="sigmoid"))

model.compile(loss=keras.losses.binary_crossentropy
            , metrics=["accuracy"], optimizer="adam")
model.fit(X_train, y_train, batch_size=256
          , validation_data = (X_test, y_test)
          , epochs=10
          , verbose = 1
          , callbacks=[tensor_board])

model.summary()
print("test accuracy: ",  model.evaluate(X_test, y_test)[1])

Train on 3428901 samples, validate on 1469530 samples
Epoch 1/10
3428901/3428901 [==============================] - 32s 9us/step - loss: 0.0447 - acc: 0.9900 - val_loss: 0.0258 - val_acc: 0.9930
Epoch 2/10
3428901/3428901 [==============================] - 32s 9us/step - loss: 0.0073 - acc: 0.9983 - val_loss: 0.0208 - val_acc: 0.9929
Epoch 3/10
3428901/3428901 [==============================] - 31s 9us/step - loss: 0.0071 - acc: 0.9984 - val_loss: 0.0220 - val_acc: 0.9926
Epoch 4/10
3428901/3428901 [==============================] - 32s 9us/step - loss: 0.0071 - acc: 0.9984 - val_loss: 0.0218 - val_acc: 0.9926
Epoch 5/10
3428901/3428901 [==============================] - 32s 9us/step - loss: 0.0070 - acc: 0.9984 - val_loss: 0.0176 - val_acc: 0.9938
Epoch 6/10
3428901/3428901 [==============================] - 32s 9us/step - loss: 0.0070 - acc: 0.9984 - val_loss: 0.0189 - val_acc: 0.9930
Epoch 7/10
 169472/3428901 [>.............................] - ETA: 33s - loss: 0.0068 - acc: 0.9984